<a href="https://colab.research.google.com/github/ValdazoAmerico/NER-HUGGINGFACE/blob/main/HUGGINGFACE_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 39.3MB/s 
     |████████████████████████████████| 901kB 47.8MB/s 
     |████████████████████████████████| 3.3MB 29.3MB/s 


In [ ]:
pip install tensorflow

In [ ]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 245kB 24.8MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 122kB 45.0MB/s 
     |████████████████████████████████| 245kB 50.1MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=acebad51eb1b5afce52fadda9d3e906dafbe275636817ba4c2b734e99a8eb3af
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/My Drive/keras_model_ner/"

In [ ]:
import pickle
pickle_in = open(path + 'tuplas_antecedentes_18_junio.pkl',"rb")
sentences = pickle.load(pickle_in)

In [ ]:
palabras = []
for k in sentences:
  frases = []
  for m in k:
    frases.append(str(m[0]))
  palabras.append(frases)

In [ ]:
ner = []
for k in sentences:
  frases = []
  for m in k:
    frases.append(m[1])
  ner.append(frases)

In [ ]:
dic = {'ner_tags': ner, 'tokens': palabras}

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(dic)

In [ ]:
df

,ner_tags,tokens
0,"[O, O, B-antecedentes, O, O, O, O]","[control, clinico, cefalea, solicito, mnp, fnp..."
1,"[O, O, O, O, B-antecedentes, O, O, O]","[control, clinico, dl, p, tbq, leve, sol, rutina]"
2,"[B-antecedentes, O, O, O, O, O, O, O, O, O, O,...","[hta, bajo, tx, mx, ok, registors, border, lin..."
3,"[O, O, O]","[paciente, que, consultando]"
4,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[dice, que, desde, 72, hs, despus, de, un, pre..."
...,...,...
253572,"[O, O, O, O, O, O, B-antecedentes, O, O, O, O,...","[infecciones, urinarias, a, repetición\r\r, \r..."
253573,"[O, O, O, O, O, O, O, O, O]","[concurre, para, recetas, de, medicaciòn, anti..."
253574,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[hto, 35, hb, 10, 9g, dl, vcm, 76, 9, hcm, 24,..."
253575,"[O, O, O, O, B-antecedentes, B-antecedentes, O...","[ecg, rs, fc, 70, brd, hbai, sin, signos, de, ..."


In [ ]:
labels_tag = []
for x in ner:
  for y in x:
    labels_tag.append(y)
dic = {}    
for e, i in enumerate ( list (set (labels_tag))):
  dic[i] = e

In [ ]:
int_tag_list = []
for x in ner:
  int_tag =[]
  for y in x:
    int_tag.append (dic[y])
  int_tag_list.append(int_tag)   

In [ ]:
dic = {'ner_tags': int_tag_list, 'tokens': palabras}

In [ ]:
df = pd.DataFrame.from_dict(dic)

In [ ]:
df.head()

,ner_tags,tokens
0,"[0, 0, 2, 0, 0, 0, 0]","[control, clinico, cefalea, solicito, mnp, fnp..."
1,"[0, 0, 0, 0, 2, 0, 0, 0]","[control, clinico, dl, p, tbq, leve, sol, rutina]"
2,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[hta, bajo, tx, mx, ok, registors, border, lin..."
3,"[0, 0, 0]","[paciente, que, consultando]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[dice, que, desde, 72, hs, despus, de, un, pre..."


In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-multilingual-uncased"
batch_size = 16

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
datasets = dataset.train_test_split(test_size=0.1)

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['asintomatico', 'ta', 'controlada', 'eco', 'bidi', 'ddvi', '43', 'dsvi', '24', 'siv', '9', '1', 'ppvi', '9', '1', 'ai', '44', 'raor', '38', 'fey', '65', 'fsvi', 'y', 'motilidad', 'parietal', 'normal', 'alteracion', 'de', 'la', 'relajacion']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'as', '##int', '##oma', '##tic', '##o', 'ta', 'control', '##ada', 'eco', 'bid', '##i', 'dd', '##vi', '43', 'ds', '##vi', '24', 'si', '##v', '9', '1', 'pp', '##vi', '9', '1', 'ai', '44', 'rao', '##r', '38', 'fey', '65', 'f', '##s', '##vi', 'y', 'mo', '##ti', '##lid', '##ad', 'par', '##ie', '##tal', 'normal', 'alter', '##acion', 'de', 'la', 're', '##la', '##ja', '##cion', '[SEP]']


In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(aligned_labels) 
# len(tokenized_input["input_ids"]))

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 14397, 25099, 3348, 2080, 10768, 3549, 5740, 4583, 1037, 29657, 2891, 15578, 3736, 5099, 8743, 6132, 2050, 1061, 22939, 20915, 5555, 4372, 23746, 2080, 9530, 8823, 3630, 4135, 2140, 1061, 4372, 7911, 18098, 4014, 23363, 2050, 11498, 14925, 2290, 12667, 15030, 10975, 1014, 2321, 25416, 21253, 14412, 23270, 21736, 2229, 8254, 2128, 2721, 10446, 9530, 9686, 11263, 2121, 6844, 27424, 11261, 2139, 9706, 8486, 10446, 1061, 2345, 21335, 10446, 7987, 2271, 3540, 14017, 28775, 3406, 12621, 2121, 102], [101, 17151, 15781, 102], [101, 14397, 25099, 2139, 4805, 1037, 29657, 2891, 8254, 2012, 2278, 6986, 12898, 12863, 2891, 2139, 12324, 2319, 7405, 2053, 7367, 16454, 10980, 2053, 9530, 17421, 2080, 2139, 21628, 22684, 6544, 1051, 19960, 5555, 10446, 2229, 21469, 2361, 23310, 2063, 24260, 6369, 2890, 10427, 8787, 3672, 2063, 28667, 25099, 3672, 2063, 2123, 2080, 6369, 2890, 5292, 3401, 2184, 22939, 2015, 11320, 20265, 2139, 8840, 12731, 2389, 2025, 2050, 18484, 2319, 9793, 2222,

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

In [ ]:
# model2.save_pretrained(path_tf)

In [ ]:
from transformers import TFDistilBertForTokenClassification, AutoConfig, AutoModelForTokenClassification,  TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=3)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.2",
  "vocab_size": 30522
}

https://huggingface.co/distilbert-base-uncased/resolve/ma

storing https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
creating metadata file for /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

In [ ]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
metric = load_metric("seqeval")

In [ ]:
label_list = ['O', 'B-antecedentes', 'I-antecedentes']

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens.
***** Running training *****
  Num examples = 228219
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 14264


Epoch,Training Loss,Validation Loss
1,0.005100,0.005455


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-ner/checkpoint-1000
Configuration saved in test-ner/checkpoint-1000/config.json
Model weights saved in test-ner/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-ner/checkpoint-1500
Configuration saved in test-ner/checkpoint-1500/config.json
Model weights saved in test-ner/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-1500/special_tokens_map.json
Sav

TrainOutput(global_step=14264, training_loss=0.013126678468134052, metrics={'train_runtime': 2214.6902, 'train_samples_per_second': 103.048, 'train_steps_per_second': 6.441, 'total_flos': 8219222623841922.0, 'train_loss': 0.013126678468134052, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens.
***** Running Evaluation *****
  Num examples = 25358
  Batch size = 16


{'epoch': 1.0,
 'eval_loss': 0.005454836413264275,
 'eval_runtime': 75.8124,
 'eval_samples_per_second': 334.483,
 'eval_steps_per_second': 20.907}

In [ ]:
import numpy as np
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens.
***** Running Prediction *****
  Num examples = 25358
  Batch size = 16


{'antecedentes': {'f1': 0.9990947397710359,
  'number': 1049935,
  'precision': 0.9995833766322912,
  'recall': 0.9986065804073585},
 'overall_accuracy': 0.9983559991352983,
 'overall_f1': 0.9990947397710359,
 'overall_precision': 0.9995833766322912,
 'overall_recall': 0.9986065804073585}

In [ ]:
path = "/content/gdrive/My Drive/NER_HUGGINGFACE/"

In [ ]:
trainer.save_model(path)

Saving model checkpoint to /content/gdrive/My Drive/NER_HUGGINGFACE/
Configuration saved in /content/gdrive/My Drive/NER_HUGGINGFACE/config.json
Model weights saved in /content/gdrive/My Drive/NER_HUGGINGFACE/pytorch_model.bin
tokenizer config file saved in /content/gdrive/My Drive/NER_HUGGINGFACE/tokenizer_config.json
Special tokens file saved in /content/gdrive/My Drive/NER_HUGGINGFACE/special_tokens_map.json


In [ ]:
config = TFAutoModelForTokenClassification.from_pretrained(path)
model2 = TFAutoModelForTokenClassification.from_config(config)

OSError: ignored

# PREDICT

In [ ]:
test_sentence = " '* HIPERTENSION ARTERIAL de más de 30 años de evolución. \r\r\n* HIPERTROFIA VENTRICULAR IZQUIERDA. \r\r\n* Esclerosis valvular aórtica, insuficiencia mitral leve. \r\r\n* EXTRASISTOLIA VENTRICULAR ASILADA monofmorfa. \r\r\n* GASTRITIS en 1999 (FEDA). \r\r\n* HISTERECTOM-A TOTAL en 1968 por MIOMATOSIS UTERINA. \r\r\n* CIRUG-A DE CADERA IZQUIERDA  (fractura) con colocación de placa de titanio en 1995. \r\r\n* HIPOACUSIA PERCETPTIVA bilateral, OTOESCLEROSIS IZQUIERDA. \r\r\n* Piel: Verrugas seborreicas en  mejillas , cuello. Manchas actinicas  en cara.  Seborrea del cuero cabelludo.\r\r\n* Olvidos irrelevantes, fue medicada con memantine en dosis de 20 mg pero lo suspendió porque le daba cefaleas. Tiene una RM (atendida en FLENI) que no mostraba alteraciones relevantes. \r\r\n* Medicación: AMLODIPINA 5 mg (Pelmec 5) diarios; SIMVASTATINA 10 mg (Labistatin) uno por día. No toma AAS por antecedentes de gastritis. \r\r\n* DESCENSO DE 12 kg de PESO en 2008 por presunta DEPRESION (no la atendió ningún especialista y el cuadro no parece tan claro). Vive con su marido; dos hijos biológicos y 1 adoptivo (problemas de aprendizaje).\r\r\n* CATARATAS BILATERALES. \r\r\n* Doctora en filososfía. Ama de casa. Jubilada. Lee mucho. \r\r\n* No alergias conocidas. \r\r\n* No tabaco ni alcohol. refiere dlp y IAM. tiene cataratas y depresión, tiene HTA y FUMA"

In [ ]:
test_sentence2 = " '* HIPERTENSION ARTERIAL de más de 30 años de evolución. \r\r\n* HIPERTROFIA VENTRICULAR IZQUIERDA. \r\r\n* Esclerosis valvular aórtica, insuficiencia mitral leve. \r\r\n* EXTRASISTOLIA VENTRICULAR ASILADA monofmorfa. \r\r\n* GASTRITIS en 1999 (FEDA). \r\r\n* HISTERECTOM-A TOTAL en 1968 por MIOMATOSIS UTERINA. \r\r\n* CIRUG-A DE CADERA IZQUIERDA  (fractura) con colocación de placa de titanio en 1995. \r\r\n* HIPOACUSIA PERCETPTIVA bilateral, OTOESCLEROSIS IZQUIERDA. \r\r\n* Piel: Verrugas seborreicas en  mejillas , cuello. Manchas actinicas  en cara.  Seborrea del cuero cabelludo.\r\r\n* Olvidos irrelevantes, fue medicada con memantine en dosis de 20 mg pero lo suspendió porque le daba cefaleas. Tiene una RM (atendida en FLENI) que no mostraba alteraciones relevantes. \r\r\n* Medicación: AMLODIPINA 5 mg (Pelmec 5) diarios; SIMVASTATINA 10 mg (Labistatin) uno por día. No toma AAS por antecedentes de gastritis. \r\r\n* DESCENSO DE 12 kg de PESO en 2008 por presunta DEPRESION (no la atendió ningún especialista y el cuadro no parece tan claro). Vive con su marido; dos hijos biológicos y 1 adoptivo (problemas de aprendizaje).\r\r\n* CATARATAS BILATERALES. \r\r\n* Doctora en filososfía. Ama de casa. Jubilada. Lee mucho. \r\r\n* No alergias conocidas. \r\r\n* No tabaco ni alcohol. refiere dlp y IAM. tiene cataratas y depresión, tiene HTA y FUMA"

In [ ]:
tokenized_inputs = tokenizer(test_sentence2.split(), truncation=True, is_split_into_words=True)

In [ ]:
tokenized_inputs

NameError: ignored

In [ ]:
dic = {'input_ids': [tokenized_inputs.input_ids]}

In [ ]:
dataset2 = Dataset.from_dict(dic)

In [ ]:
predictions, labels, _ = trainer.predict(dataset2)
predictions = np.argmax(predictions, axis=2)

tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"])
mensaje_original = tokenizer.convert_tokens_to_string(tokens)
print("NER Tags:", predictions[0])
print("Tokens:",tokens)
print("Mensaje original:",mensaje_original)

***** Running Prediction *****
  Num examples = 1
  Batch size = 16


NER Tags: [1 1 1 0 0 0 0 2 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 2 2 2 1 1 1 1 1 1 1 0 0
 0 1 1 1 1 1 1 1 0 0 0 0 2 2 1 1 1 1 0 0 0 2 2 2 1 1 1 1 1 1 1 1 1 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1
 1]
Tokens: ['[CLS]', "'", '*', 'hip', '##ert', '##ens', '##ion', 'arterial', 'de', 'mas',

# LOAD MODEL

In [ ]:
from transformers import TFDistilBertForTokenClassification, AutoTokenizer

In [ ]:
model_checkpoint = "distilbert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.2",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe

In [ ]:
path_tf = '/content/gdrive/My Drive/NER_HUGGINGFACE_TF/'

In [ ]:
model = TFDistilBertForTokenClassification.from_pretrained(path_tf)

loading configuration file /content/gdrive/My Drive/NER_HUGGINGFACE_TF/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/gdrive/My Drive/NER_HUGGINGFACE/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.2",
  "vocab_size": 30522
}

loading weights file /content/gdrive/My Drive/NER_HUGGINGFACE_TF/tf_model.h5
Some layers from the model checkpoint at /content/gdrive/My Drive/NER_HUGGINGFACE_TF/ were not used whe

In [ ]:
from transformers import pipeline
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

In [ ]:
nlp("tengo tos")

[{'end': 5,
  'entity_group': 'LABEL_1',
  'score': 0.9999653,
  'start': 0,
  'word': 'tengo'},
 {'end': 9,
  'entity_group': 'LABEL_0',
  'score': 0.99958634,
  'start': 6,
  'word': 'tos'}]

In [ ]:
filter = ['cefalea',
          'cefaleas',
          'tos',
          'diarrea',
          'disnea',
          'astenia',
          'mareos',
          'edema',
          'lumbalgia',
          'cervicalgia',
          'colico',
          'placas',
          'fatiga',
          'epigastralgia',
          'faringitis',
          'dolores',
          'placa',
          'esencial',
          'higado',
          'faringitis aguda',
          'diarrea guda',
          'gastroenteritis aguda',
          'astenica']

In [ ]:
import re
def preprocess(txt):
  text = txt.lower()
  text = text.translate(str.maketrans("áéíóú", "aeiou"))
  text = re.sub(r'[^\w\s]',' ',txt)
  return text

In [ ]:
def predict(txt):
  prediction = nlp(txt)
  return prediction

In [ ]:
def postprocess(output):
   antecedentes = []
   for w in output:
    if w['entity_group'] == "LABEL_0":
      if w['word'] not in filter:
        antecedentes.append(w['word'])
    if w['entity_group'] == "LABEL_2":
        if len(antecedentes) > 0:
          last = antecedentes.pop()
          antecedentes.append(last + " " + w['word'])
        
   antecedentes = list(set(antecedentes))
   for a in antecedentes:
     if a in filter:
       antecedentes.remove(a)
   return antecedentes

In [ ]:
text1 = "hipertension arterial aguda hipertension arterial hipertension aguda faringitis aguda"
text_largo = " '* HIPERTENSION ARTERIAL de más de 30 años de faringitis aguda evolución. \r\r\n* HIPERTROFIA VENTRICULAR IZQUIERDA. \r\r\n* Esclerosis valvular aórtica, insuficiencia mitral leve. \r\r\n* EXTRASISTOLIA VENTRICULAR ASILADA monofmorfa. \r\r\n* GASTRITIS en 1999 (FEDA). \r\r\n* HISTERECTOM-A TOTAL en 1968 por MIOMATOSIS UTERINA. \r\r\n* CIRUG-A DE CADERA IZQUIERDA  (fractura) con colocación de placa de titanio en 1995. \r\r\n* HIPOACUSIA PERCETPTIVA bilateral, OTOESCLEROSIS IZQUIERDA. \r\r\n* Piel: Verrugas seborreicas en  mejillas , cuello. Manchas actinicas  en cara.  Seborrea del cuero cabelludo.\r\r\n* Olvidos irrelevantes, fue medicada con memantine en dosis de 20 mg pero lo suspendió porque le daba cefaleas. Tiene una RM (atendida en FLENI) que no mostraba alteraciones relevantes. \r\r\n* Medicación: AMLODIPINA 5 mg (Pelmec 5) diarios; SIMVASTATINA 10 mg (Labistatin) uno por día. No toma AAS por antecedentes de gastritis. \r\r\n* DESCENSO DE 12 kg de PESO en 2008 por presunta DEPRESION (no la atendió ningún especialista y el cuadro no parece tan claro). Vive con su marido; dos hijos biológicos y 1 adoptivo (problemas de aprendizaje).\r\r\n* CATARATAS BILATERALES. \r\r\n* Doctora en filososfía. Ama de casa. Jubilada. Lee mucho. \r\r\n* No alergias conocidas. \r\r\n* No tabaco ni alcohol. refiere dlp y IAM. tiene cataratas y depresión, tiene HTA y FUMA"
text_largo2 = "'* Asmático desde los 9 meses de edad.\r\r\n* Presentó neumonías a los 9 meses, 12 años, 20 años y a los 36 años, complicada esta última con absceso pulmonar y septicemia.\r\r\n* Apendicectomía y amigdalectomía en infancia. Hernioplastia inguinal en edad adulta. No medicamentos.                              PADECE  COLITIS ULCEROSA. \r\r\n Cirugía de C-NCER DE ESOFAGO el 31 de marzo de 2004. \r\r\n* Biopsia de MASA ABDOMINAL (oct. 2006): CARCINOMA POCO DIFERENCIADO (no se determina primitivo). En seguimiento para tratamiento oncológico. \r\r\n* EPISODIOS SINCOPALES desde agosto de 2004, estudiado con Holter sp, ECG sp. se sugirió una dudosa hipoglucemia, TAC cerebral normal, Doppler de vasos del cuello (set. 2005)  normal. TEST DE MOVIMIENTO OSCILATORIO CORPORAL (TILT TEST) del 1 de nov. de 2004: Negativo (sin medicación). \r\r\n* GLAUCOMA de ANGULO ABIERTO (puede usar antiespasmódicos). Tiene antecedentes de IAM y presentó síncope'"
text_largo3 = '64 años(2005)Litiasis biliar no operada.cesarea 3. Noidulectomia mamaria./r/DLP sin TTO.Osteoporosis sin TTO.'

In [ ]:
model_input = preprocess(text_largo)
model_output = predict(model_input)
postprocess(model_output)

['extrasistolia ventricular',
 'hipoacusia',
 'hta',
 'esclerosis',
 'hipertrofia ventricular',
 'cataratas',
 'gastritis',
 'insuficiencia mitral',
 'fractura',
 'dlp',
 'miomatosis',
 'iam',
 'hipertension arterial aguda',
 'depresion']

In [ ]:
def text2ant(txt):
  model_input = preprocess(txt)
  model_output = predict(model_input)
  return postprocess(model_output)


# TEST CON LISTA DE ANTECEDENTES

In [ ]:
import pandas as pd
df2 = pd.read_csv('message.txt', delimiter="/t", header=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
row = {0: text_largo }
df2 = df2.append(row ,ignore_index=True)

In [ ]:
row = {0: text_largo2 }
df2 = df2.append(row ,ignore_index=True)

In [ ]:
df2['ner_huggingface'] = df2[0].apply(text2ant)

In [ ]:
df2

,0,ner_huggingface
0,"[' HTA EN TTO CON LOSARTAN 50 MG / DIA - OMEPRAZOL 20 MG / DIA - LEVOTIROXINA 100 MCG - 75 MCG LOS LUNES , MIERCOLES Y VIERNES - ATENOLOL 50 MG / DIA AN...","[colecistectomia, polipo colonico, hta]"
1,"'64 años(2005)Litiasis biliar no operada.Cesareas 3. Noidulectomia mamaria.DLP sin TTO.Osteoporosis sin TTO.',","[litiasis biliar, osteoporosis, dlp]"
2,'COLECISTECTOMIA. OBESIDAD. INCONTINENCIA URINARIA DE ESFUERZO. ...,"[hta, colecistectomia, acv, sordera, menopausia, incontinencia urinaria, obesidad]"
3,'* Anemia en 1999. Consume predominantemente vegetales. \r\r\n* Medicación habitual: no . \r\r\n* Alergias: no. \r\r\n* Cirugías: No. No internaciones. \r\r\n* No tabaco. No alcohol. \r\r\n* Pract...,"[alergias, rinitis, anemia]"
4,"'cx de rodilla derecha,colpuril y corenitec,cx prostata y de hernia inguinal .',","[hernia inguinal, cx]"
5,"'* Hiperplasia nodular difusa de tiroides. * Psoriasis. * Taquicardias habituales. * Ex TBQ hasta hace 10 a.',","[tbq, taquicardias, psoriasis]"
6,"'* Asmático desde los 9 meses de edad.\r\r\n* Presentó neumonías a los 9 meses, 12 años, 20 años y a los 36 años, complicada esta última con absceso pulmonar y septicemia.\r\r\n* Apendicectomía y ...","[glaucoma, apendicectomia, carcinoma, hipoglucemia, cirugia, neumonias, asmatico, hernioplastia inguinal]"
7,"'HTA medicada con medicacion homeopatica, Reacciones alergica en piel a repeticion Tres embarazos 3 abortos provocados Colecistectom...","[colecistectomia, alergica, hta, insomnio]"
8,'* Hipercolesterolemia. \r\r\n* Dolores articulares a predominio de manos y cuello. \r\r\n* Medicación: antiinflamatorios sn. \r\r\n* Cambios evacuatorios en año 2000. \r\r\n* Colecistectomía. \r\...,"[colecistectomia, rinitis, hipercolesterolemia, alergia]"
9,"'HTA medicada con enalapril 10 mgdia FA medicada con Atlansil 5-7 , Cx por prolapso de matriz apendicectomia Zoster en el mes de enero Actualmente medicada con Bago b...","[prolapso, apendicectomia, hta, gastritis, colon, cx, fa]"


In [ ]:
df2.to_csv(path_tf + "df_antecedentes_ner_huggingface.csv")

In [ ]:
pd.set_option("max_colwidth", 200)

In [ ]:
path_tf

'/content/gdrive/My Drive/NER_HUGGINGFACE_TF/'

In [ ]:
df2[0][14]

"'TOMA CRONICAMENTE  TOTAL MAGNESIANO  \\r\\r\\nNEVUS  EN REGION FRONTAL  DESDE HACE 2 AÑOS  SIN SINTOMAS . \\r\\r\\nREALIZA CONTROLES GINECOLOGICOS ANIVEL PARTICULAR . \\r\\r\\nEcografÝa abdominal con hÝgado graso en a±o 2003. \\r\\r\\nASM-TICA. ',"